# Monadic Parsing

Based on Graham Huttons's "Programming in Haskell". This notebook showcases how monads
can be used to implement parsers.

"A parser is a program that takes a string of characters as input, and produces
some form of tree that makes the syntactic structure of the string explicit." (Graham Hutton)

In [1]:
using Pkg
Pkg.activate(".")
using PartialFunctions

  Activating project at `~/MEGA/EMAP/Julia_Tutorials/CategoryTheory`


In [2]:
struct Parser{a}
    #string -> [(a,string)]
    p::Function
end

In [3]:
parse(p::Parser, s::String) = p.p(s)
f(x::String) = [(1,x)]
p = Parser{Int}(f)
parse(p,"ok")

1-element Vector{Tuple{Int64, String}}:
 (1, "ok")

In [4]:
# item::Parser{Char}
item = Parser{Char}(
        (inp -> inp == "" ? [] : [(inp[1], string(inp[2:end]))])
        )

parse(item, "")
parse(item, "abc")

1-element Vector{Tuple{Char, String}}:
 ('a', "bc")

In [5]:
fmap(f::Function, p::Parser{T}) where T = Parser{T}(
        inp -> begin
            res = parse(p, inp)
            res == [] ? [] : [(f $ res[1][1]
                ,res[1][2])]
        end
    )

fmap(::Tuple{}, p::Parser{T}) where T = Parser{T}(
        inp -> begin
            res = parse(p, inp)
            res == [] ? [] : [(res[1][1] <| ()
                ,res[1][2])]
        end
    )

fmap (generic function with 2 methods)

In [6]:
parse(fmap(uppercase, item),"")
parse(fmap(uppercase, item),"abc")
fmap((),fmap(uppercase, item)).p("abc")

1-element Vector{Tuple{Char, String}}:
 ('A', "bc")

In [27]:
pure(v::T) where T = Parser{T}(inp -> [(v,inp)])
⊗(pg::Parser{<: Function}, px::Parser) = Parser{Function}(
        inp -> begin
            res = parse(pg, inp)
            println(res)
            if res == []
                return []
            end
            g,out = res[1]
            parse(fmap(g,px),out)
        end
    )

⊗ (generic function with 2 methods)

In [32]:
p = pure((x,y,z)->(x,z)) ⊗ item ⊗ item ⊗ item

p.p("ok")

[(var"#69#70"(), "ok")]
Tuple{PartialFunctions.PartialFunction{var"#69#70", Tuple{Char}, NamedTuple{(), Tuple{}}}, String}[(#69('o', ...), "k")]
Tuple{PartialFunctions.PartialFunction{var"#69#70", Tuple{Char, Char}, NamedTuple{(), Tuple{}}}, String}[(#69('o', 'k', ...), "")]


Any[]

In [ ]:
η = pure
bind(p::Parser{a},f::FunctionWrapper{Parser{b}, Tuple{a}}) where {a,b} = Parser(P{b}(
        inp -> begin
            res = parse(p, inp)
            @show res
            res == [] ? [] : parse(f(res[1][1]), res[1][2])
        end
    ))

η(1).p("ok")

In [140]:
# pure(1)
# g = FunctionWrapper{Parser{Int},Tuple{String}}(x->pure(length(x)))
# bind(pure(""),g).p("")

In [155]:
# g = FunctionWrapper{Tuple{Char,Char},Tuple{String}}(x->
pg(x,y,z) = (x,z)

# parse(pure(pg),"akg")
pure(pg).p("ok")[1][1]

pg (generic function with 1 method)

1-element Vector{Tuple{Int64, String}}:
 (1, "abc")

In [136]:
g = FunctionWrapper{Char, Tuple{Char}}(x->uppercase(x))

pure(g) ⊗ item

Parser{Char}(FunctionWrapper{Vector{Tuple{Char, String}}, Tuple{String}}(Ptr{Nothing} @0x00007eff985c7a80, Ptr{Nothing} @0x00007eff6257b370, Base.RefValue{var"#295#296"{Parser{FunctionWrapper{Char, Tuple{Char}}}, Parser{Char}}}(var"#295#296"{Parser{FunctionWrapper{Char, Tuple{Char}}}, Parser{Char}}(Parser{FunctionWrapper{Char, Tuple{Char}}}(FunctionWrapper{Vector{Tuple{FunctionWrapper{Char, Tuple{Char}}, String}}, Tuple{String}}(Ptr{Nothing} @0x00007eff985c4bb0, Ptr{Nothing} @0x00007effa4706bb0, Base.RefValue{var"#293#294"{FunctionWrapper{Char, Tuple{Char}}}}(var"#293#294"{FunctionWrapper{Char, Tuple{Char}}}(FunctionWrapper{Char, Tuple{Char}}(Ptr{Nothing} @0x00007eff985c8300, Ptr{Nothing} @0x00007effa547c0a8, Base.RefValue{var"#319#320"}(var"#319#320"()), var"#319#320"))), var"#293#294"{FunctionWrapper{Char, Tuple{Char}}})), Parser{Char}(FunctionWrapper{Vector{Tuple{Char, String}}, Tuple{String}}(Ptr{Nothing} @0x00007eff98551c00, Ptr{Nothing} @0x00007effa547c010, Base.RefValue{var"#7#8